In [1]:
import pandas as pd
from dateutil import parser
import os
import flair

In [2]:
# Import the sentiment classifier (English)
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')

2022-04-19 15:14:11,555 loading file C:\Users\20182402\.flair\models\sentiment-en-mix-distillbert_4.pt


In [5]:
# Set a path to the articles and create an empty dataframe
path = os.getcwd()+'/articles'
data = []
df = pd.DataFrame(columns=["File_Name", "Date","Sentiment", "Publisher", "Mentioned_Names", "Text"])

In [4]:
# Open the records data and save all employee names in a list
file_name = 'EmployeeRecords.xlsx'
sheet = 'Employee Records'
df_names = pd.read_excel(io=file_name, sheet_name=sheet)
names = df_names.FirstName + ' ' + df_names.LastName
names = names.to_list()

In [7]:
# Extract sentiment, publisher, mentioned names, time, and add it to a dataframe
for filename in os.listdir(path):

    with open(os.path.join(path, filename), 'r') as f: # Get all files in the folder
        x = f.read()

        # Sentiment
        article = ''
        for line in x:
            article += line.replace('\n', ' ').replace("\"", "" )
        s = flair.data.Sentence(article)
        flair_sentiment.predict(s)
        total_sentiment = s.labels

        # Mentioned Names
        article_names = []
        for person_name in names:
            if person_name in article:
                article_names.append(person_name)

        # Publisher
        lines = x.splitlines()
        length = len(lines)
        if lines[0]!='':
            publisher = lines[0]
        elif lines[2]!='':
            publisher = lines[2]
        else:
            publisher = lines[4]

        # Time
        try:
            date = parser.parse(lines[3])
        except:
            try:
                date = parser.parse(lines[6])
            except:
                try:
                    date = parser.parse(lines[7])
                except:
                    try:
                        date = parser.parse(lines[2])
                    except:
                        date = parser.parse(lines[4])
        df = df.append({"File_Name": filename,"Date":  date, "Sentiment":total_sentiment, "Publisher":publisher, "Mentioned_Names":article_names, "Text":article}, ignore_index=True)
df_files_sorted_or = df.sort_values(by='Date')

C:\Users\20182402\AppData\Local\Temp/ipykernel_20716/2060425680.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"File_Name": filename,"Date":  date, "Sentiment":total_sentiment, "Publisher":publisher, "Mentioned_Names":article_names, "Text":article}, ignore_index=True)
C:\Users\20182402\AppData\Local\Temp/ipykernel_20716/2060425680.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"File_Name": filename,"Date":  date, "Sentiment":total_sentiment, "Publisher":publisher, "Mentioned_Names":article_names, "Text":article}, ignore_index=True)
C:\Users\20182402\AppData\Local\Temp/ipykernel_20716/2060425680.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"File_Name": filen

In [8]:
# Further preprocess the data by adding some extra columns and sorting
df_files_sorted = df_files_sorted_or.reset_index()
df_files_sorted['Positive'] = [1 if str(df_files_sorted.Sentiment[i][0]).split()[0]=='POSITIVE' else 0 for i in range(len(df_files_sorted))]
df_files_sorted['Negative'] = abs(df_files_sorted['Positive']-1)
df_files_sorted['Certainty'] = [df_files_sorted.Sentiment[i][0].score for i in range(len(df_files_sorted))]
df_files_sorted['Score'] = [df_files_sorted['Certainty'][i] if df_files_sorted['Positive'][i]==1 else -1*df_files_sorted['Certainty'][i]for i in range(len(df_files_sorted))]
df_files_sorted

,index,File_Name,Date,Sentiment,Publisher,Mentioned_Names,Text,Positive,Negative,Certainty,Score
0,802,728.txt,1982-10-02 00:00:00,[POSITIVE (0.9918)],World Source,[],World Source Centrum's central role in the T...,1,0,0.991822,0.991822
1,702,638.txt,1982-10-02 00:00:00,[POSITIVE (0.9851)],International Times,[],International Times Centrum's central role in ...,1,0,0.985125,0.985125
2,232,214.txt,1982-10-03 00:00:00,[NEGATIVE (0.905)],Everyday News,[],Everyday News Central paper of the center in ...,0,1,0.905014,-0.905014
3,655,596.txt,1982-10-03 00:00:00,[POSITIVE (0.6304)],News Online Today,[],News Online Today Centrum's central role in ...,1,0,0.630370,0.630370
4,877,796.txt,1982-10-03 00:00:00,[POSITIVE (0.9747)],The Continent,[],The Continent The central role of the center ...,1,0,0.974651,0.974651
...,...,...,...,...,...,...,...,...,...,...,...
943,254,234.txt,2014-03-26 00:00:00,[NEGATIVE (0.9996)],Everyday News,[],Everyday News 2014/03/26 21 of October of...,0,1,0.999621,-0.999621
944,604,55.txt,2014-03-26 00:00:00,[NEGATIVE (0.5309)],Daily Pegasus,[],Daily Pegasus INFLAMMATION OF THE PROTESTS I...,0,1,0.530852,-0.530852
945,303,279.txt,2014-03-26 00:00:00,[POSITIVE (0.9558)],World Journal,[],World Journal The perspicacity of an expert...,1,0,0.955815,0.955815
946,756,687.txt,2014-03-26 00:00:00,[NEGATIVE (0.996)],World Journal,[],"World Journal 2014/03/26 October 21, 2013...",0,1,0.996014,-0.996014


In [9]:
# Save the dataframe in the current folder
df_files_sorted.to_csv('df_sentiment_sorted.csv', index=False)